## Google Analytics Customer Revenue Prediction Data Rewrite v2

*NOTE on v2:<br>
A separate v2 kernel was created because train_v2.csv file is 24GB and must be split into chunks and therefore this version must be sensitive to memory limitations. There are also two new columns which are nested lists and dicts. The column 'hits' is extremely large and has many such nested fields, however it has been expanded like the nested json columns albeit with some limited compromise to data integrity.*

**NEEDS A FIX: strip '\r' and other rubbish to prevent it being copied to the resultant csvs**

This kernel is merely show how to convert the train.csv and test.csv from large cumbersome with JSON columns to the lighter newtrain.csv and newtest.csv, where the nested JSON entries have been expanded (flattened) into extra columns. Since all JSON field name repetition is removed the resultant files are approximately 1GB lighter (eg. 1.5GB to 250MG for train.csv). Though no information has been lost, the resultant files are in UTF-8 format instead of ASCII.

This kernel is set up to do the conversion from original data to new data and write it to your working directory, even though dataset **"GA data with json columns"** is already included (it purposefully fails to find the new dataset and proceeds to load and convert the original one). To change this behaviour and load the new dataset immediately uncomment the line  `newdata_dir = "../input/ga-analytics-with-json-columns"` in the first code cell.

#### dtypes/type changes after reloading data
It is worth noting that a reload of the manipulated data (eg. `newtrain = pd.read_csv(newtrain_path)`) assigns more appropriate dtypes and types (to columns and elements respectively) than the first csv read. The sequence is (1) read original csv into DataFrame `train`; (2) which is then copied to `newtrain`; (3) JSON fields are added; (4) `newtrain` is written to a new csv; (5) `newtrain` is reloaded from the new csv. Only after (5) is done do we see appropriate values for many columns.
* Eg: `totals.transactionRevenue` starts out life as an **object** with elements of type **&lt;class 'float'>**, but once reloaded the dtype and type change to **float64** and **&lt;class 'numpy.float'>** respectively. The latter change was important when using `pd.sum()`.

#### dtype of fullVisitorId
Many `fullVisitorId`s have leading zeros, which are stripped when a csv is read without forcing the dtype(str). Since the example submission contains leading zeros we suppose we must keep the zeros.

#### extra stuff
After the reload, there is some extra stuff which doesn't affect the new cvs such as dropping columns filled only with NaNs. Also, there some rudimentary EDA pies.

## Data overview

_NOTE: Jupyter has removed support for :--- to left justify so have had to use `<p align="left">`_<br>
_NOTE: JSON indicates many more columns will be added once these columns (with json fields) have been expanded_<br>
_NOTE: LD indicates many more columns will be added once these columns (with lists/dicts) have been expanded_

Field|<p align="left">Description
---|:---
 fullVisitorId        |<p align="left"> A unique identifier for each user of the Google Merchandise Store
 customDimensions     |<p align="left"> (LD) Lists and Dicts with index and region info 
 channelGrouping      |<p align="left"> The channel via which the user came to the Store
 date                 |<p align="left"> The date on which the user visited the Store
 device               |<p align="left"> (JSON) The specifications for the device used to access the Store
 geoNetwork           |<p align="left"> (JSON) This section contains information about the geography of the user
 hits                 |<p align="left"> (LD) Lists and Dicts which have a lot of info on hits
 sessionId            |<p align="left"> A unique identifier for this visit to the store
 socialEngagementType |<p align="left"> Engagement type, either "Socially Engaged" or "Not Socially Engaged"
 totals               |<p align="left"> (JSON) This section contains aggregate values across the session
 trafficSource        |<p align="left"> (JSON) This section contains information about the Traffic Source from which the session originated. 
 visitId              |<p align="left"> An identifier for this session. This is part of the value usually stored as the `_utmb` cookie, uique only to user. For unique visit ID use: fullVisitorId and visitId
 visitNumber          |<p align="left"> The session number for this user. If this is the first session, then this is set to 1
 visitStartTime       |<p align="left"> The timestamp (expressed as POSIX time)
     


##### Useful websites
https://pandas.pydata.org/pandas-docs/stable/api.html#flat-file

## Initialisation
* import the usual helpful libs
* initialise path names (using multiple locations)
* read into newtrain and newtest if available, otherwise train and test

In [1]:
import os
import sys
import warnings
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
import time
import gc
import ast

warnings.filterwarnings('ignore')
orig_max_rows = pd.options.display.max_rows

def set_path(paths):
    for path in paths:
        if os.path.exists(path):
            return path

#newdata_dir = "."
newdata_dir = "../input/ga-analytics-with-json-columns"
data_dir1   = "../input/ga-customer-revenue-prediction"
data_dir2   = "../input"
newtrain_path = newdata_dir+"/newtrain_v2.csv"
newtest_path  = newdata_dir+"/newtest_v2.csv"
train_path    = set_path([data_dir1+"/train_v2.csv", data_dir2+"/train_v2.csv"])
test_path     = set_path([data_dir1+"/test_v2.csv",  data_dir2+"/test_v2.csv"])

!ls -ld $newdata_dir/*.csv $data_dir1/*.csv $data_dir2/*.csv

ls: cannot access '../input/*.csv': No such file or directory
-rwxrwxrwx 1 colin colin        7255 Oct 25 21:18  ../input/ga-analytics-with-json-columns/ga_20170601.csv
-rwxrwxrwx 1 colin colin        1157 Oct 25 21:20  ../input/ga-analytics-with-json-columns/ga_20170601_fromtrain.csv
-rwxrwxrwx 1 colin colin     1212669 Oct 25 21:36  ../input/ga-analytics-with-json-columns/ga_20170801_20180430_rev_gt0.csv
-rwxrwxrwx 1 colin colin   222497940 Oct 28 21:32  ../input/ga-analytics-with-json-columns/newtest.csv
-rwxrwxrwx 1 colin colin   244350270 Oct 28 21:31  ../input/ga-analytics-with-json-columns/newtrain.csv
-rwxrwxrwx 1 colin colin  6298048094 Nov 29 03:18  ../input/ga-analytics-with-json-columns/newtrain_v2.csv
-rwxrwxrwx 1 colin colin    18412503 Nov  8 09:51  ../input/ga-analytics-with-json-columns/submission10.csv
-rwxrwxrwx 1 colin colin    19478856 Nov  8 09:57  ../input/ga-analytics-with-json-columns/submission11.csv
-rwxrwxrwx 1 colin colin    18651683 Nov  8 10:34  ../input/

In [2]:
def load_new_or_orig(newpath, path, chunksize=None):
    new = None
    orig = None
    if os.path.exists(newpath):
        new = pd.read_csv(newpath, dtype={'fullVisitorId': 'str', 'visitId': 'str'})
        print ("loaded",newpath)
    elif os.path.exists(path):
        orig = pd.read_csv(path, dtype={'fullVisitorId': 'str', 'visitId': 'str'}, chunksize=chunksize)
        print ("readied",path,"to be read in chunks of",chunksize)
    else:
        print ("ERROR: loaded nothing")
    return new, orig

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
// Kaggle fails on the above lines

<IPython.core.display.Javascript object>

## Generic data parsing functions
* summary: columns for "describe" and default ("basic") should be self-explanatory
* _recursive_lds: function for ld_normalize to create cols from nested lists and dicts
* ld_normalize: like json_normalize, but for nested lists and dicts
* is_json: guess a JSON col
* is_ld: guess a LD (list/dict) col
* get_json_cols: get all JSON cols in df
* get_ld_cols: get all LD (list/dict) cols in df
* effect_na: convert certain strings to np.nan
* get_json: get a specific JSON col and rewrite using data found
* get_ld: get a specific LD (list/dict) col and rewrite using data found 
* expand_json: expand all JSON cols
* expand_ld: expand all LD (list/dict) cols

In [4]:
def summary(df, info="describe", cols=None):
    if info == "describe":
        headings=['Null','dType','Type','MinMax','Mean','Std','Skew','Unique','Examples']
    elif info == "revenue":
        headings=['Null','nNull','nRevs','nNull%','nRevs%','dType','Type','Unique','Examples']    
        nrevs = df['totals.transactionRevenue'].count()
    else:
        headings=['Null','dType','Type','Unique','Examples']

    if cols == None:
        cols = df.columns

    nrows = df.shape[0]
    if (nrows > orig_max_rows):
        pd.set_option('display.max_rows', nrows)

    print('DataFrame shape',df.shape)
    sdf = pd.DataFrame(index=cols, columns=headings)
    for col in cols:
        sys.stdout.write('.')
        sys.stdout.flush()
        sdf['Null'][col]     = df[col].isna().sum()
        sdf['Unique'][col]   = len(list(df[col].unique()))
        sdf['dType'][col]    = df[col].dtypes
        sdf['Type'][col]     = "-" if df[col].notna().sum() == 0 else type(df[col].dropna().iloc[0])
        sdf['Examples'][col] = "-" if df[col].notna().sum() == 0 else list(df[col].unique())
        if info == "describe":
            if 'float' in str(df[col].dtype) or 'int' in str(df[col].dtype):
                sdf['MinMax'][col] = str(round(df[col].min(),2))+'/'+str(round(df[col].max(),2))
                sdf['Mean'][col]   = df[col].mean()
                sdf['Std'][col]    = df[col].std()
                sdf['Skew'][col]   = df[col].skew()
        elif info == "revenue":
            sdf['nNull'][col] = df[col].count()
            sdf['nRevs'][col] = df.groupby(col)['totals.transactionRevenue'].count().sum()
            sdf['nNull%'][col] = round(sdf['nNull'][col] / (nrows/100), 1)
            sdf['nRevs%'][col] = round(sdf['nRevs'][col] / (nrevs/100), 1)
    return sdf.fillna('-')

# _recursive_lds and ld_normalize: recursive lists and dicts string to DataFrame
# The array lists would normally have their own col name, ie. hits.list1.blah = "blah",
# but there are some cells over 30000 chars long (eg. 'hits' column in row where
# fullVisitorId = 309482894121265066) and giving unique col names to lists mean
# literally thousands of columns so I keep the array lists generic and append values.
# The dict names and unique. This keeps the col numbers down.
# Debug example: print (debugdf.loc[debugidx,'fullVisitorId'],col,"list"+str(ki))

# IMPROVE: d[col] += "."+val
# - don't append if val == d[col]
# - strip '\r' and other rubbish - there's a lot of it
def _recursive_lds(val, col, d={}, debugdf=None, debugidx=None):
    if type(val) is dict:
        for k,v in val.items():
            _recursive_lds(v, col+"."+k, d=d, debugdf=debugdf, debugidx=debugidx)
    elif type(val) is list:
        for ki,v in enumerate(val):
            #_recursive_lds(v, col+".list"+str(ki), d=d, debugdf=debugdf, debugidx=debugidx)
            _recursive_lds(v, col+".list", d=d, debugdf=debugdf, debugidx=debugidx)
    else:
        try:
            d[col] += "."+val
        except:
            d[col] = val
    return d
   
def ld_normalize(df, col):
    ad = []
    for idx,elem in enumerate(df[col]):
        ad.append(_recursive_lds(ast.literal_eval(elem), col, d={}, debugdf=df, debugidx=idx))
    return pd.DataFrame(ad)

# need to improve is_json and is_ld
#def is_json(df, col):
#    return df[col].dtype == object and df[col].str.contains(r'{".*:.*"}').sum() > 0
#def is_ld(df, col):
#    return df[col].dtype == object and df[col].str.contains(r'{\'.*:.*\'}').sum() > 0

def get_json_cols(df):
    #return [col for col in df.columns if is_json(df, col)]
    return ['device', 'geoNetwork', 'totals', 'trafficSource']

def get_ld_cols(df):
    #return [col for col in df.columns if is_ld(df, col)]
    return ['customDimensions', 'hits']

def effect_na(df):
    for col in df.columns:
        for na_str in ['not available in demo dataset', '(not provided)', '(not set)']:
            df[col].replace(na_str, np.nan, inplace=True, regex=True)
    return df

def get_json(df, col):
    df[col] = df[col].apply(json.loads)    # do normalize separately or it will use just one column
    df[col] = df[col].apply(lambda x: {} if pd.isnull(x) else x)
    jdf = pd.io.json.json_normalize(df[col]).add_prefix(col+'.')
    jdf = effect_na(jdf)
    return jdf

def get_ld(df, col):
    lddf = ld_normalize(df, col)  # don't use df[col] like json_normalize - easier to debug
    lddf = effect_na(lddf)
    return lddf

# since df will be a chunk from a csv file, do all drops (of json converts cols) when the file is finished
def expand_json(df):
    for col in get_json_cols(df):
        df = pd.concat([df.reset_index(drop=True), get_json(df, col)], axis=1, sort=False)
    return df

def expand_ld(df):
    for col in get_ld_cols(df):
        df = pd.concat([df.reset_index(drop=True), get_ld(df, col)], axis=1, sort=False)
    return df

In [5]:
#newtrain, train = load_new_or_orig(newtrain_path, train_path, chunksize=10)
##newtrain, train = load_new_or_orig(newtrain_path, "./newtrain_v2_test.csv", chunksize=100)
#for df in train:
#    print (get_json_cols(df))
#    print (get_ld_cols(df))
#    col='hits'
#    #df[col] = df[col].apply(ast.literal_eval)
#    newdf = get_ld(df, col)
#    break
#    #newdf
#print (newdf.shape)
#newdf.T
##newdf.to_csv("./zz.csv", index=False)

## Create wider data frames using nested json and list/dict entries
* create newtrain/newtest and then save them as csvs; they will be smaller since we have removed duplicate json and list/dict keys

In [6]:
def freemem(df):
    del df
    gc.collect()
    
def used_mb(df):
    return df.memory_usage().sum() / (1024 * 1024)

def check_extra_cols(orig, curr):
    extra = list(sorted(set(curr) - set(orig)))
    if (len(extra) > 0):
        print ("ERROR: extra col(s) detected:",extra)

# writing in segments may mean extra cols later on, so a header is written each time
def read_write_df_in_segments(largefile, newdata_path, writemb=1024, debugbreak=0):
    newdata = pd.DataFrame()
    rows = 0
    cols = None
    for dataseg in largefile:
        sys.stdout.write('.')
        sys.stdout.flush()
        dataseg = expand_json(dataseg)
        dataseg = expand_ld(dataseg)
        newdata = pd.concat([newdata, dataseg], axis=0, sort=False)
        print (used_mb(newdata),"MB memory used")
        freemem(dataseg)

        if (debugbreak and rows >= debugbreak):
            break

        if used_mb(newdata) >= writemb:
            newdata.drop(columns=get_json_cols(newdata), inplace=True)
            newdata.drop(columns=get_ld_cols(newdata), inplace=True)
            if cols is None:
                print (used_mb(newdata),"MB memory used so creating csv")
                cols = newdata.columns
                newdata.to_csv(newdata_path, index=False, encoding="utf-8", mode='w', columns=cols)
            else:
                print (used_mb(newdata),"MB memory used so appending to csv")
                check_extra_cols(cols, newdata.columns)
                newdata.to_csv(newdata_path, index=False, encoding="utf-8", mode='a', columns=cols, header=False)
            rows = rows + newdata.shape[0]
            freemem(newdata)
            newdata = pd.DataFrame()

    if newdata.shape[0] > 0:
        newdata.drop(columns=get_json_cols(newdata), inplace=True)
        newdata.drop(columns=get_ld_cols(newdata), inplace=True)
        print ("writing last {:d} rows to csv".format(newdata.shape[0]))
        check_extra_cols(cols, newdata.columns)
        newdata.to_csv(newdata_path, index=False, encoding="utf-8", mode='a', columns=cols, header=False)
    print ("finished writing {:s}, rows = {:d}".format(newdata_path, rows))


chunksize=50000; debugbreak=0; writemb=256

#print (time.ctime(),"converting/loading training data v2:")
#newtrain, train = load_new_or_orig(newtrain_path, train_path, chunksize=chunksize)
#if newtrain is None:
#    read_write_df_in_segments(train, newtrain_path, writemb=writemb, debugbreak=debugbreak)
#    freemem(train)
#    newtrain, train = load_new_or_orig(newtrain_path, train_path, chunksize=0)

print ("")
print (time.ctime(),"converting/loading test data v2:")
newtest, test = load_new_or_orig(newtest_path, test_path, chunksize=chunksize)
if newtest is None:
    read_write_df_in_segments(test, newtest_path, writemb=writemb, debugbreak=debugbreak)
    freemem(test)
    newtest, test = load_new_or_orig(newtest_path, test_path, chunksize=0)
    
print ("")
print (time.ctime(),"finished")


Thu Nov 29 07:45:13 2018 converting/loading test data v2:
readied ../input/ga-customer-revenue-prediction/test_v2.csv to be read in chunks of 50000
.58.7940979004 MB memory used
.118.350982666 MB memory used
.177.526473999 MB memory used
.236.701965332 MB memory used
.295.877456665 MB memory used
284.433364868 MB memory used so creating csv
.58.7940979004 MB memory used
.118.350982666 MB memory used
.177.526473999 MB memory used
.179.407071114 MB memory used
writing last 151589 rows to csv
finished writing ../input/ga-analytics-with-json-columns/newtest_v2.csv, rows = 250000
loaded ../input/ga-analytics-with-json-columns/newtest_v2.csv

Thu Nov 29 08:09:01 2018 finished


In [ ]:
summary(newtrain, info="basic")

In [7]:
summary(newtest, info="basic")

DataFrame shape (401590, 149)
.....................................................................................................................................................

,Null,dType,Type,Unique,Examples
channelGrouping,1,object,<class 'str'>,9,"[Organic Search, Direct, Affiliates, Referral,..."
date,0,object,<class 'int'>,171,"[20180511, 20180602, 20180819, 20180918, 20180..."
fullVisitorId,0,object,<class 'str'>,296531,"[7460955084541987166, 460252456180441002, 3461..."
socialEngagementType,1,object,<class 'str'>,2,"[Not Socially Engaged, nan]"
visitId,0,object,<class 'str'>,393182,"[1526099341, 1526064483, 1526067157, 152610755..."
visitNumber,1,float64,<class 'numpy.float64'>,387,"[2.0, 166.0, 4.0, 1.0, 3.0, 26.0, 28.0, 10.0, ..."
visitStartTime,0,object,<class 'int'>,393693,"[1526099341, 1526064483, 1526067157, 152610755..."
device.browser,1,object,<class 'str'>,63,"[Chrome, Internet Explorer, Safari (in-app), E..."
device.browserSize,401590,float64,-,1,-
device.browserVersion,401590,float64,-,1,-


## Rewrite and Reload
* if the directory is writable, write newtrain and newtest if they don't already exist
* reload back into newtrain and newtest which allows pandas.read_csv to assign more appropriate types (see explanation in heading)
<br>*NOTE: python3's default ecoding is "utf-8", but explicitly set it so we know what we're getting*

In [ ]:
# newtrain.csv is only 250MB (from 1.5GB) because we've removed the json repetition
# newtest.csv is only 230MB (from 1.3GB) because we've removed the json repetition

if os.access(newdata_dir, os.W_OK):
    if not os.path.exists(newtrain_path):
        #newtrain.to_csv(newtrain_path, index=False, encoding="utf-8")
        #print ("wrote", newtrain_path)
        newtrain = pd.read_csv(newtrain_path, dtype={'fullVisitorId': 'str', 'visitId': 'str'})
        print ("reloaded newtrain")
    if not os.path.exists(newtest_path):
        #newtest.to_csv(newtest_path, index=False, encoding="utf-8")
        #print ("wrote", newtest_path)
        newtest = pd.read_csv(newtest_path, dtype={'fullVisitorId': 'str', 'visitId': 'str'})
        print ("reloaded newtest")
else:
    print (newdata_dir, "is not writable")

In [ ]:
!echo Quick validation of rows:
!sed -n '$=' $newtrain_path
#!sed -n '$=' $train_path
print ("1708338")
!sed -n '$=' $newtest_path
#!sed -n '$=' $test_path
print ("401590")

## Simplifying newtrain and newtest
* start with the obvious: if a column has no data for newtrain OR newtest, it is useless to both (we'll drop these)
* there two columns not common to both: **totals.transactionRevenue** (we need); **trafficSource.campaignCode** (we'll just ignore this)

In [ ]:
def get_unused(df):
    rows = df.shape[0]
    ddf = df.isna().sum()
    return list(ddf[ddf >= rows].index)
    
droplist = get_unused(newtrain)
for dropcol in get_unused(newtest):
    if dropcol not in droplist:
        droplist.append(dropcol)

print ("dropping these columns from both newtrain and newtest:")
print (droplist)
newtrain.drop(columns=droplist, inplace=True)
newtest.drop(columns=droplist, inplace=True)

In [8]:
#print (get_unused(newtest))
unused_cols_test = ['device.browserSize',
                    'device.browserVersion',
                    'device.flashVersion',
                    'device.language',
                    'device.mobileDeviceBranding',
                    'device.mobileDeviceMarketingName',
                    'device.mobileDeviceModel',
                    'device.mobileInputSelector',
                    'device.operatingSystemVersion',
                    'device.screenColors',
                    'device.screenResolution',
                    'geoNetwork.cityId',
                    'geoNetwork.latitude',
                    'geoNetwork.longitude',
                    'geoNetwork.networkLocation',
                    'trafficSource.adwordsClickInfo.criteriaParameters',
                    'hits.list.contentGroup.contentGroup4',
                    'hits.list.contentGroup.contentGroup5',
                    'hits.list.page.searchCategory',
                    'hits.list.product.list.productCouponCode']

['device.browserSize', 'device.browserVersion', 'device.flashVersion', 'device.language', 'device.mobileDeviceBranding', 'device.mobileDeviceMarketingName', 'device.mobileDeviceModel', 'device.mobileInputSelector', 'device.operatingSystemVersion', 'device.screenColors', 'device.screenResolution', 'geoNetwork.cityId', 'geoNetwork.latitude', 'geoNetwork.longitude', 'geoNetwork.networkLocation', 'trafficSource.adwordsClickInfo.criteriaParameters', 'hits.list.contentGroup.contentGroup4', 'hits.list.contentGroup.contentGroup5', 'hits.list.page.searchCategory', 'hits.list.product.list.productCouponCode']


In [ ]:
summary(newtrain, info="basic")

In [ ]:
summary(newtest, info="basic")

## EDA of newtrain data
* **totals.transactionRevenue**: NaN vs >0  (imbalance is obvious without a graph)

In [ ]:
zrows = newtrain['totals.transactionRevenue'].size
nrows = newtrain['totals.transactionRevenue'].dropna().size
print ("NaN =", zrows, "; >0 =", nrows)
#plt.bar([1,2], [zrows, nrows])
#plt.ylabel('Rows', fontsize=15)
#plt.xticks([1,2], ["NaN", ">0"], fontsize=15, rotation=0)
#plt.title("totals.transactionRevenue", fontsize=15);

## Pie chart functions
* col_by_col_count: eg. (df, OS, session) gets session instances counted per OS (so total OS usage)
* col_by_col_count: eg. (df, OS, revenue) gets revenue instances counted per OS
* col_by_col_sum: eg. (df, OS, revenue) gets revenue total per OS
* myautopct: pie is messy with 0.0 and 0.1 percent markings, so set a threshold
* mypie: common options

In [ ]:
def col_by_col_count(df, col1, col2, threshold=0):
    return df.groupby([col1]).count()[col2].apply(lambda x: (x if x>threshold else np.nan)).dropna()

def col_by_col_sum(df, col1, col2, threshold=0):
    return df.groupby([col1]).sum(numeric_only=True)[col2].apply(lambda x: (x if x>threshold else np.nan)).dropna()

def myautopct(pct):
    return ('%.2f' % pct) if pct > 2 else ''

def mypie(df, title, angle=0):
    # autopct='%1.1f%%'
    # textprops={'size': 'small'}  (Kaggle python (3.6.6) + libs didn't recognise this)
    df.plot(kind='pie', figsize=(5, 5), radius=1.2, startangle=angle, autopct=myautopct, pctdistance=0.8,
        rotatelabels=False, legend=True, explode=[0.02]*df.size);
    plt.title(title, weight='bold', size=14, x=2.0, y=-0.01);
    plt.xlabel('')
    plt.ylabel('')
    plt.legend(bbox_to_anchor=(2.5, 1.0), ncol=2, fontsize=10, fancybox=True, shadow=True);

## Pie deductions
* OSs: 19 in total, and we've removed negligable sales on Nintendo etc.
* OS split: Windows the clear winner, no suprises there. 10 OSs below the threshold and so not represented here.
* OS by transaction instance: people buy more on a Mac it seems
* OS by transaction sum: more or less the above, except people spend bigger amounts when using Chrome OS and Windows

In [ ]:
newtrain['device.operatingSystem'].astype(str).unique()

In [ ]:
df = col_by_col_count(newtrain, 'device.operatingSystem', 'sessionId', threshold=100)
mypie(df, 'OS prevalence', angle=100)

In [ ]:
df = col_by_col_count(newtrain, 'device.operatingSystem', 'totals.transactionRevenue', threshold=100)
mypie(df, 'OS prevalence by revenue instances', angle=0)

In [ ]:
df = col_by_col_sum(newtrain, 'device.operatingSystem', 'totals.transactionRevenue', threshold=100)
mypie(df, 'OS prevalence by revenue sum', angle=10)